# Astra DB - Querying Existing Vector Data

This sample assumes that there's already a Vector DB created, and a table with Vector data in it. And shows how to connect to that database and run a ANN query on that table

## Setup

In [ ]:
pip install cassandra-driver

## Get DB Connection

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [ ]:
import os
from getpass import getpass

try:
    from google.colab import files
    IS_COLAB = True
except ModuleNotFoundError:
    IS_COLAB = False

In [ ]:
# Your database's Secure Connect Bundle zip file is needed:
if IS_COLAB:
    print('Please upload your Secure Connect Bundle zipfile: ')
    uploaded = files.upload()
    if uploaded:
        astraBundleFileTitle = list(uploaded.keys())[0]
        ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
    else:
        raise ValueError(
            'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
        )
else:
    # you are running a local-jupyter notebook:
    ASTRA_DB_SECURE_BUNDLE_PATH = input("Please provide the full path to your Secure Connect Bundle zipfile: ")

ASTRA_DB_APPLICATION_TOKEN = getpass("Please provide your Database Token ('AstraCS:...' string): ")
ASTRA_DB_KEYSPACE = input("Please provide the Keyspace name for your Database: ")
ASTRA_DB_VECTOR_TABLE = input("Please provide the name of your vector table: ")

In [ ]:
# Don't mind the "Closing connection" error after "downgrading protocol..." messages,
# it is really just a warning: the connection will work smoothly.
cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        "token",
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)

session = cluster.connect()
keyspace = ASTRA_DB_KEYSPACE

## Let's explore the table

In [ ]:
from cassandra.query import SimpleStatement
query = SimpleStatement(
    f"""
    SELECT *
    FROM {ASTRA_DB_KEYSPACE}.{ASTRA_DB_VECTOR_TABLE} LIMIT 2;
    """
    )

results = session.execute(query)
top_answers = results._current_rows

for row in top_answers:
  print(row)

## ANN Search

In [ ]:
query_vector=[0.010530042462050915, -0.00725283520296216, 0.008213164284825325, -0.03604927659034729, -0.0014564432203769684, 0.0042946189641952515, -0.030488770455121994, -0.008428063243627548, 0.0014749111142009497, -0.023356815800070763, 0.006185056641697884, 0.025855015963315964, 0.005365754943341017, 0.015432422049343586, -0.02155703864991665, 0.025210319086909294, 0.04268427938222885, -0.011530664749443531, 0.0007248638430610299, -0.0054664891213178635, -0.040293529629707336, 0.005234801210463047, 0.004553169012069702, -0.008884723298251629, 0.010375583544373512, 0.025653546676039696, -0.00019611616153270006, -0.02710411511361599, 0.010429308749735355, -0.0010837280424311757, 0.0401860810816288, -0.0004700911813415587, -0.023961219936609268, 0.018964821472764015, -0.00044154992792755365, -0.010174116119742393, -0.017970914021134377, -0.00965030025690794, 0.024095531553030014, -0.0027718590572476387, 0.03191247582435608, 0.007011074107140303, -0.009482410736382008, 0.00893173273652792, -0.02428356744349003, 0.02475365810096264, -0.006809606682509184, -0.00863624643534422, 0.002021392108872533, 0.019905004650354385, 0.007897531613707542, -0.006044029723852873, -0.017554547637701035, -0.018441004678606987, -0.011342628858983517, 0.006480542942881584, -0.03441067412495613, 0.0030774183105677366, -0.014250478707253933, 0.012987948022782803, 0.014384790323674679, 0.020334800705313683, -0.030354458838701248, 0.015620458871126175, -0.01289392914623022, 0.005063553806394339, -0.013807249255478382, 0.004479297436773777, -0.008925016969442368, 0.016560640186071396, 0.022013697773218155, 0.019260307773947716, -0.003767444984987378, 0.013605781830847263, -0.013901268132030964, 0.002229575300589204, -0.03282579407095909, -0.016977006569504738, -0.008918301202356815, 0.006211919244378805, 0.019461775198578835, -0.03030073456466198, 0.015257816761732101, 0.02018705941736698, 0.008609384298324585, 0.0068666888400912285, -0.017232200130820274, 0.015969669446349144, -0.014304202981293201, 0.00646711140871048, -0.010140538215637207, 0.005560507066547871, 0.011839582584798336, 0.005577296018600464, -0.02930682711303234, 0.012443984858691692, 0.018252968788146973, 0.018252968788146973, -0.0077497889287769794, -0.000502409937325865, 0.016009962186217308, -0.012450700625777245, -0.011228463612496853, -0.004066288936883211, -0.01469370722770691, -0.007326706778258085, -0.0001217200406244956, -0.017447099089622498, 0.028232334181666374, -0.019730398431420326, -0.014774294570088387, -0.0037875918205827475, 0.0057552591897547245, -0.033148143440485, -0.009885345585644245, -0.00398905947804451, -0.012296242639422417, -0.004757994320243597, -0.020845185965299606, -0.0022430066019296646, 0.03416891396045685, 0.004878874868154526, 0.04647187143564224, -0.020818322896957397, 0.021046653389930725, 0.00036767846904695034, -0.027144407853484154, -0.0013859295286238194, -0.019287168979644775, -0.016090549528598785, 0.04279172793030739, 0.013733377680182457, 0.024444742128252983, 0.01060391403734684, -0.00449608638882637, 0.021812230348587036, -0.027144407853484154, 0.02457905374467373, 0.000209547346457839, -0.0022681900300085545, -0.0009259117650799453, 0.021073516458272934, -0.006890193559229374, 0.011288903653621674, -0.01974382996559143, 0.039380211383104324, 0.03422263637185097, 0.0043752058409154415, -0.02185252495110035, -0.013632643967866898, 0.013122259639203548, -0.019233444705605507, 0.010657638311386108, 0.02927996590733528, 0.007037936709821224, 0.010006226599216461, -0.012437270022928715, -0.004291261080652475, -0.016117412596940994, -0.0037640873342752457, -0.013928130269050598, -0.006080965045839548, -0.0024797311052680016, -0.00618841452524066, -0.005040049087256193, 0.029145652428269386, 0.05380529537796974, 0.01403557974845171, 0.005698176566511393, 0.0012230764841660857, -0.0008020930690690875, 0.0018048143247142434, -0.03441067412495613, 0.01756797917187214, 0.01900511421263218, 0.012652168050408363, 0.016023393720388412, 0.01719190552830696, 0.006457038223743439, 0.002049933420494199, -0.010711363516747952, 0.01733964867889881, -0.003027051454409957, 0.03129464015364647, -0.022456927224993706, 0.00015939028526190668, 0.006809606682509184, -0.001032521715387702, 0.009294373914599419, -0.013471470214426517, 0.01201418787240982, 0.025129731744527817, -0.0366671122610569, -0.020697442814707756, -0.6640373468399048, -0.015257816761732101, 0.016788970679044724, -0.0017510897014290094, 0.04461836814880371, 0.02807115949690342, 0.023182211443781853, 0.006527551915496588, -0.015418990515172482, 0.007105092518031597, 0.003194941207766533, -0.005063553806394339, 0.007863953709602356, 0.008501934818923473, -0.004982966464012861, -0.032369136810302734, 0.011235179379582405, -0.014304202981293201, -0.030703669413924217, 0.01997216045856476, -0.02866213023662567, 0.019864710047841072, -0.016936713829636574, -0.03287952020764351, 0.0029414277523756027, 0.028823304921388626, 0.015418990515172482, -0.026835491880774498, -0.018911097198724747, 0.02534463070333004, -0.0001105973424273543, 0.02903820388019085, 0.013095397502183914, 0.0036499223206192255, 0.05474547669291496, 0.0022631532046943903, -0.02725185826420784, 0.001971025252714753, 0.0066383592784404755, 0.03261089697480202, -0.035619478672742844, -0.01314240600913763, 0.0007450105622410774, 0.0020432176534086466, -0.013357304967939854, -0.0022950523998588324, 0.028823304921388626, 0.034921057522296906, 0.006849899888038635, -0.007494596764445305, -0.008710118010640144, 0.02613706886768341, -0.016708383336663246, 0.008501934818923473, -0.0007021987112239003, -0.0024797311052680016, 0.02960231341421604, -0.003921903669834137, 0.014156459830701351, 0.016950145363807678, -0.018642473965883255, 0.02651314251124859, -0.030032111331820488, -0.0036499223206192255, -0.020146764814853668, 0.01485488098114729, -0.003794307354837656, 0.005903002340346575, 0.018736490979790688, -0.025331199169158936, 0.0037741607520729303, 0.0065107629634439945, -0.02572070248425007, 0.0014010396553203464, -0.0032704914920032024, 0.021422727033495903, 0.017930621281266212, -0.01672181487083435, -0.011745563708245754, 0.020898910239338875, -0.0036197020672261715, -0.005029975436627865, 0.024189548566937447, -0.011376206763088703, 0.027090683579444885, 0.016386035829782486, -0.022080853581428528, -0.008891439065337181, -0.0017393373418599367, -0.011913454160094261, 0.026499710977077484, 0.027560774236917496, 0.005298599135130644, -0.02581472136080265, -0.012826773338019848, 0.005681387614458799, -0.037150636315345764, -0.029441138729453087, -0.01742023602128029, -0.004801645874977112, 0.01860217936336994, 0.03634476289153099, 0.006097753997892141, 0.012987948022782803, 0.017769446596503258, 0.002259795553982258, -0.0002182566240662709, 0.01075165718793869, 0.014465377666056156, -0.030032111331820488, -0.008985457010567188, -0.014237047173082829, -0.0006996803567744792, -0.014062441885471344, 0.011302335187792778, -0.02294044941663742, 0.009495842270553112, -0.015580165199935436, -0.001491700066253543, -0.026096776127815247, 0.02807115949690342, -0.016171136870980263, 0.02115410380065441, -0.009099622257053852, -0.02663402259349823, 0.02445817179977894, -0.019220013171434402, -0.027507049962878227, -0.030757393687963486, -0.0006342033739201725, -0.00346188573166728, 0.017756015062332153, -0.006473827175796032, 0.0011651546228677034, 0.008367623202502728, 0.002180887386202812, -0.007904247380793095, -0.026620592921972275, 0.006040671840310097, -0.010509896092116833, -0.009442117065191269, 0.010711363516747952, 0.010268134996294975, -0.0041770958341658115, -0.0022648321464657784, -0.01974382996559143, -0.0067088729701936245, -0.007306559942662716, -0.005080342758446932, -0.0014556037494912744, -0.017030732706189156, -0.012853636406362057, -0.011282188817858696, 0.016009962186217308, 0.003915187902748585, -0.007581899408251047, -0.004240894224494696, -0.027802536264061928, -0.011497086845338345, -0.0030874917283654213, -0.001269246218726039, 0.04905065894126892, 0.0002765982935670763, -0.021140672266483307, 0.005087058059871197, -0.026298243552446365, 0.006678652483969927, 0.011423216201364994, -0.021113809198141098, -0.023034468293190002, 0.009643584489822388, -0.014828018844127655, 0.01403557974845171, 0.024135824292898178, -0.019340895116329193, 0.011631399393081665, -0.003911830019205809, -0.004378563724458218, 0.005647809710353613, -0.0031613633036613464, 0.01120831724256277, -0.005063553806394339, 0.01531154103577137, 0.012094774283468723, 0.011356059461832047, -0.001367461634799838, 0.010335290804505348, 0.031026018783450127, -0.01029499713331461, 0.025358062237501144, 0.005359039641916752, 0.026701178401708603, 0.006601423490792513, 0.003670068923383951, -0.010436024516820908, -0.0029397488106042147, -0.0026845564134418964, 0.0006178340991027653, 0.014935468323528767, 0.008703402243554592, -0.0014564432203769684, -0.00874369591474533, 0.007017789874225855, -0.00764905521646142, 0.0035760507453233004, -0.007373716216534376, -0.004768067505210638, -0.0070580835454165936, -0.02291358821094036, 0.044188570231199265, 0.0006992606213316321, -0.004959462210536003, 0.0011475261999294162, 0.013055103830993176, -0.003280564909800887, 0.0057653323747217655, 0.015418990515172482, 0.01766199804842472, -0.0016721815336495638, -0.00011636855197139084, -0.003492105985060334, 0.008898154832422733, 0.006977496203035116, -0.00859595276415348, -0.008146008476614952, 0.0069976430386304855, 0.008898154832422733, 0.030515633523464203, -0.005960084497928619, -0.025895308703184128, -0.014707138761878014, 0.0002870913885999471, 0.01030842773616314, -0.009113053791224957, 0.007743073161691427, 0.013189415447413921, -0.017648566514253616, -0.017769446596503258, 0.03306755796074867, 0.001814887742511928, -0.007454303093254566, 0.0025217034853994846, 0.0003752335032913834, 0.0021003002766519785, 0.008098999969661236, 0.01724563166499138, 0.035270269960165024, 0.008166155777871609, -0.013793818652629852, 0.00318318884819746, -0.012316389009356499, -0.013646075502038002, -0.018857372924685478, 0.017232200130820274, -0.01605025678873062, -0.040293529629707336, 0.01939461939036846, 0.0023554926738142967, 0.004113297909498215, 0.017702290788292885, 0.01157095842063427, 0.006896909326314926, 0.004828508011996746, -0.0052784522995352745, -0.01107400469481945, 0.01991843432188034, 0.006074249744415283, -0.013216277584433556, 0.012094774283468723, -0.005144140683114529, -0.01637260429561138, 0.008280320093035698, 0.016735246405005455, 0.012443984858691692, 0.028850167989730835, 0.02968290075659752, 0.010885968804359436, -0.013175983913242817, -0.018749922513961792, 0.00401927949860692, -0.03411518782377243, -0.027359306812286377, 0.0097510339692235, 0.01625172421336174, 0.0134916165843606, -0.0046438295394182205, -0.018696198239922523, -0.015848789364099503, -0.005644451826810837, 0.0220002681016922, -0.007984834723174572, 0.0023571716155856848, -0.018897665664553642, 0.017285924404859543, 0.026150500401854515, -0.0036734268069267273, 0.027413031086325645, 0.011752279475331306, -0.001381732290610671, -0.01932746358215809, 0.002483088755980134, -0.0011181454174220562, -0.004788214340806007, -0.020804893225431442, 0.016909850761294365, -0.010818812996149063, 0.0009158383472822607, -0.03121405467391014, 0.015499577857553959, -0.022040560841560364, 0.0005464809946715832, -0.015392128378152847, -0.009865199215710163, -0.03951452299952507, 0.012155215255916119, 0.013585635460913181, -0.03384656459093094, 0.0038983989506959915, 0.04786871373653412, 0.0005292723071761429, -0.012182077392935753, -0.0214092954993248, -0.004163664765655994, 0.01592937484383583, 0.09251394122838974, 0.01756797917187214, -0.0024192905984818935, 0.008629530668258667, -0.004569957964122295, 0.02155703864991665, -0.002343740314245224, -0.007353569380939007, 0.02056313119828701, -0.017285924404859543, 0.003918545786291361, -0.0015378696843981743, -0.004143517930060625, -0.004146875813603401, 0.010214409790933132, 0.008770558051764965, 0.00477478327229619, -0.010892684571444988, -0.02499542012810707, -0.02026764489710331, -0.013008094392716885, 0.0253714919090271, 0.041287437081336975, 0.0026123637799173594, 0.012632021680474281, 0.016466623172163963, 0.012417122721672058, 0.01044274028390646, -0.005976873449981213, -0.012652168050408363, -0.002162419492378831, -0.0005049282917752862, -0.007595330476760864, -0.0015009340131655335, -0.007722926791757345, 0.030488770455121994, -0.0018216032767668366, 0.04104567691683769, -0.026889216154813766, 0.01498919352889061, 0.01545928418636322, 0.026204226538538933, -0.0025166666600853205, -0.023988081142306328, -0.002417611889541149, 0.011624683625996113, -0.028205471113324165, 0.020348232239484787, -0.010039804503321648, -0.008569090627133846, -0.0038278852589428425, -0.006463753990828991, 0.011396353133022785, -0.0030119414441287518, -0.009865199215710163, 0.012242517434060574, -0.0240552369505167, -0.018494730815291405, -0.004254325293004513, -0.018172381445765495, -0.008602668531239033, -0.016036825254559517, -0.007991550490260124, 0.013860974460840225, -0.003586124163120985, -0.03056935779750347, 0.0019861352629959583, -0.04015921801328659, -4.252987309882883e-06, 0.004203958436846733, 0.009724171832203865, -0.022510651499032974, -0.036882009357213974, -0.00883099902421236, 0.04405425861477852, 0.02475365810096264, 0.0009301090030930936, -0.005281810183078051, 0.006430175621062517, 0.02678176574409008, 0.019703535363078117, -0.03188561275601387, 0.004539737477898598, -0.011020280420780182, -0.015324972569942474, -0.00582913076505065, 0.005842561833560467, 0.007004358805716038, 0.026942940428853035, 0.008790705353021622, 0.030381321907043457, -0.0008453246555291116, 0.024162687361240387, -0.017406804487109184, -0.011624683625996113, 0.0001889808481791988, 0.009542850777506828, 0.023826908320188522, -0.005537002347409725, -0.030166422948241234, 0.0016805760096758604, 0.009354814887046814, -0.00664507457986474, -0.0019408049993216991, -5.6190587201854214e-05, 0.015432422049343586, 0.002132199238985777, -0.010932978242635727, -0.010247987695038319, -0.01701730117201805, 0.013874405063688755, -0.00041216923273168504, 0.009892061352729797, 0.006782744079828262, 0.02326279878616333, 0.03951452299952507, -0.006074249744415283, 0.016950145363807678, -0.01830669306218624, 0.002792005892843008, -0.01803806982934475, -0.04198585823178291, 0.02777567319571972, 0.017299355939030647, -0.006873404607176781, 0.003911830019205809, -0.02534463070333004, -0.025801289826631546, -0.020012453198432922, 0.004402067977935076, -0.01701730117201805, 0.0024629419203847647, -0.0037607294507324696, -0.00783037580549717, -0.035243406891822815, -0.01660093478858471, 0.0007198271341621876, 0.013713231310248375, -0.011302335187792778, -0.005701534450054169, -0.030515633523464203, 0.014102735556662083, 0.006712230388075113, -0.03513595834374428, -0.021315276622772217, -0.03911158815026283, -0.0044423616491258144, 0.009455548599362373, -0.00950255710631609, 0.010174116119742393, 0.005493351258337498, -0.008246742188930511, -0.026499710977077484, 0.008441494777798653, -0.005419479683041573, -0.011758995242416859, -0.006148121319711208, 0.008925016969442368, 0.03422263637185097, 0.0369625985622406, 0.014384790323674679, 0.009972648695111275, 0.024135824292898178, -0.003851389978080988, -0.015096643008291721, -0.013209561817348003, 0.007427440490573645, -0.006211919244378805, -0.008837713859975338, 0.009824905544519424, 0.015029486268758774, 0.00525159016251564, 0.008871291764080524, 0.01836041919887066, -0.011893306858837605, 0.010368868708610535, 0.003142895409837365, -0.018642473965883255, -0.03024701029062271, -0.0057653323747217655, 0.0045733158476650715, -0.001992850797250867, -0.01930060051381588, 0.006430175621062517, 0.003864821046590805, -0.015472715720534325, 0.0018602180061861873, 0.015875650569796562, -0.0023806761018931866, -0.01622486114501953, 0.056625839322805405, -0.007689348421990871, 0.024887969717383385, -0.014102735556662083, 0.010127107612788677, -0.031026018783450127, -0.01045617088675499, -0.0017065989086404443, -0.01434449665248394, 0.011188169941306114, 0.011812719516456127, 0.013028241693973541, -0.008481788448989391, -0.0022933734580874443, 0.0024226484820246696, 0.014143029227852821, -0.029253102838993073, -0.01257158163934946, 0.0019408049993216991, -0.031670715659856796, 0.017111318185925484, -0.015942806378006935, -0.023477695882320404, -0.014922037720680237, 0.002886024070903659, 0.005258305463939905, -0.0256669782102108, -0.008011696860194206, -0.003471959149464965, -0.03497478365898132, -0.002385712694376707, 0.039998043328523636, 0.027936847880482674, 0.0016998832579702139, 0.028447231277823448, 0.008501934818923473, -0.002007960807532072, -0.007508027832955122, 0.0025032355915755033, -0.011873160488903522, -0.013505048118531704, 0.02651314251124859, 0.021073516458272934, 0.005392617546021938, 0.001409434131346643, 0.0037405826151371002, -0.0054765623062849045, -0.009542850777506828, 0.004130086861550808, 0.009965932928025723, -0.0008872970938682556, 0.010368868708610535, -0.024189548566937447, -0.022779274731874466, -0.021812230348587036, 0.03484047204256058, 0.0014488882152363658, -0.006426818203181028, -0.01363935973495245, -0.026069913059473038, -0.005735112354159355, 0.011732133105397224, -0.011960462667047977, 0.009280943311750889, 0.004381921608000994, -0.014975761994719505, 0.006077607627958059, -0.02774881012737751, 0.004391994792968035, 0.009327951818704605, -0.0038178120739758015, 0.0214092954993248, -0.014572826214134693, 0.013108828105032444, -0.01236339844763279, -0.01107400469481945, 0.008253457956016064, 0.0006908661453053355, 0.013317011296749115, 0.028877029195427895, -0.011973894201219082, -0.01120831724256277, -0.005879497621208429, -0.0023571716155856848, 0.0034820325672626495, -0.0363716259598732, -0.029414277523756027, 0.0013061818899586797, -0.018843941390514374, -0.004130086861550808, 0.01500262413173914, 0.001843429054133594, -0.012927507981657982, 0.0023101624101400375, -0.003992417361587286, -0.0011063931742683053, -0.00834747590124607, 0.012813342735171318, 0.0011676729191094637, -0.007051367778331041, -0.018763354048132896, -0.007675917353481054, -0.0004986324347555637, 0.01050318032503128, 0.02472679689526558, -0.015969669446349144, 0.013914698734879494, 0.009468979202210903, -0.031375229358673096, 0.001992850797250867, 0.023679165169596672, 0.0032318769954144955, 0.00034648238215595484, 0.006356304511427879, -0.004959462210536003, -0.020670581609010696, 0.0060339560732245445, 0.002442795317620039, -0.010489748790860176, -0.037177495658397675, -0.0021657771430909634, 0.0008360907668247819, -0.012558150105178356, -0.013276718556880951, 0.0003221383667550981, -0.003210051218047738, 0.0025989327114075422, -0.013733377680182457, -0.01531154103577137, -0.020455682650208473, -0.02440444752573967, 0.007581899408251047, 0.01580849476158619, 0.00899217277765274, 0.010019658133387566, -0.02126155234873295, 0.002763464581221342, -0.010395730845630169, -0.014626551419496536, -0.02220173552632332, -0.03064994513988495, 0.027936847880482674, -0.02011990174651146, 0.011604536324739456, -0.0236120093613863, 0.017151612788438797, -0.015177229419350624, -0.010053236037492752, 0.001547103631310165, -0.029468001797795296, 0.0019894931465387344, 0.021785369142889977, 0.005459773354232311, -0.005731754470616579, 0.001997887622565031, -0.02130184695124626, -0.019958728924393654, -0.0012339893728494644, -0.026069913059473038, -0.01813208870589733, -0.008454925380647182, 0.03782219439744949, 0.01868276670575142, -0.017138181254267693, -0.03782219439744949, 0.005355681758373976, -0.03529713302850723, -0.02044225111603737, 0.0054765623062849045, 0.021758506074547768, 0.010174116119742393, 0.005936580244451761, -0.022738981992006302, 0.014062441885471344, 0.013928130269050598, -0.0023538137320429087, -0.005772048141807318, -0.0009099622257053852, -0.013981854543089867, -0.018736490979790688, 0.023061329498887062, -0.015378696843981743, -0.04015921801328659, -7.093339809216559e-05, 0.012034334242343903, 0.013753524981439114, -0.0067995330318808556, 0.010530042462050915, 0.0077497889287769794, -0.000558653031475842, -0.020630287006497383, 0.02370602637529373, 0.027668224647641182, 0.014008717611432076, 0.00020461558597162366, -0.00692377146333456, 0.010630776174366474, 0.013914698734879494, 0.008193017914891243, -0.017433667555451393, 0.009220502339303493, 0.019193151965737343, -0.0019005114445462823, 0.0004919168422929943, -0.005543718114495277, -0.00533217703923583, -0.0003097565204370767, -0.02229575254023075, -0.022698689252138138, -0.013303580693900585, -0.012410406954586506, 0.011846297420561314, -0.01560702733695507, -0.04276486486196518, 0.005936580244451761, 0.011604536324739456, -0.0003225581021979451, -0.003767444984987378, -0.000808808661531657, -0.019233444705605507, -0.01076508779078722, 0.0017510897014290094, 0.023746320977807045, -0.025881877169013023, -0.01838728040456772, -0.011275473050773144, -0.001964309485629201, -0.025774428620934486, 0.011537380516529083, 0.0009662052616477013, 0.012108205817639828, 0.0067290193401277065, 0.013511763885617256, 0.014210185036063194, 0.008219880051910877, -0.003223482286557555, 0.020401956513524055, -0.011873160488903522, 0.01060391403734684, 0.019542362540960312, -0.010281565599143505, -0.03467929735779762, -0.0015227596741169691, 0.013941561803221703, 0.0038010228890925646, 0.004942672792822123, 0.24412506818771362, 0.01991843432188034, 0.008569090627133846, 0.052838247269392014, 0.01751425489783287, 0.004667333792895079, 0.019193151965737343, -0.005392617546021938, -0.006853257771581411, -0.009495842270553112, 0.004982966464012861, 0.02259123884141445, -0.04467209428548813, 0.004005848430097103, 0.0012801589909940958, -0.014828018844127655, -0.026002757251262665, -0.0008457443909719586, 0.0171650443226099, -0.0030656661838293076, 0.0033158217556774616, -0.02103322185575962, 0.00041091005550697446, -0.0019794197287410498, 0.02410896122455597, 0.017554547637701035, -0.028312919661402702, 0.012007472105324268, 0.024659641087055206, 0.009939070791006088, -0.009791327640414238, -0.012155215255916119, 0.010885968804359436, 0.010630776174366474, -0.026553437113761902, -0.013545341789722443, 0.005503424443304539, 0.014451946131885052, 0.0233165230602026, 0.0021976763382554054, 0.0033393262419849634, 0.011389637365937233, -0.015499577857553959, -0.022215167060494423, -0.008569090627133846, 0.002931354334577918, -0.001563053228892386, -0.02118096500635147, 0.007655770517885685, 0.024391015991568565, -0.022953880950808525, -0.01298123225569725, 0.030112698674201965, 0.020670581609010696, -0.005177718587219715, -0.019367756322026253, -0.006413386669009924, -0.009260796010494232, -0.014438514597713947, -0.015889082103967667, 0.012235801666975021, 0.033658530563116074, 0.0020835113245993853, 0.04093822464346886, -0.010348721407353878, 0.013444608077406883, -0.04093822464346886, 0.011913454160094261, 0.002538492437452078, -0.019193151965737343, -0.010583767667412758, -0.0006098593585193157, -0.00740057835355401, 0.007696064189076424, -0.01798434555530548, -0.009724171832203865, 0.02408210001885891, 0.04598834738135338, 0.022228596732020378, 0.008481788448989391, -0.004828508011996746, 0.012047765776515007, -0.021019792184233665, 0.0016344062751159072, -0.01419675350189209, -0.038413166999816895, 0.017621703445911407, -0.014223615638911724, 0.008340761065483093, -0.00868325587362051, 0.004808361176401377, 0.002931354334577918, -0.01544585358351469, -0.015472715720534325, -0.007581899408251047, 0.00538254389539361, -0.000809648132417351, 0.021610762923955917, -0.00016033467545639724, -0.007152101490646601, -0.01883050985634327, -0.013249855488538742, -0.009529420174658298, 0.021745074540376663, -0.013310296460986137, 0.029172515496611595, -0.0002728207618929446, 0.01709788851439953, 0.01766199804842472, -0.0032167667523026466, -0.007434156257659197, -0.012927507981657982, 0.009133200161159039, -0.008542228490114212, 0.03024701029062271, 0.01830669306218624, -0.0007135312771424651, -0.026378830894827843, 0.0022833000402897596, -0.010073382407426834, 0.002889381954446435, -0.0035256838891655207, -0.008716833777725697, 0.017030732706189156, -0.006020525004714727, -0.027439894154667854, -0.003263775957748294, -0.0032553814817219973, 0.011356059461832047, -0.039353348314762115, 0.023826908320188522, -0.019220013171434402, -0.008555659092962742, 0.009643584489822388, 0.007467734161764383, -0.004660618491470814, 0.011510518379509449, -0.025411786511540413, 0.0025703913997858763, -0.009280943311750889, -0.0011508839670568705, 0.016345741227269173, 0.020925773307681084, -0.004052857402712107, 0.02611020766198635, 0.008481788448989391, -0.010953124612569809, -0.00035214866511523724, -0.016587503254413605, -0.011026996187865734, -0.014747432433068752, -0.00090660440037027, -0.019878141582012177, -0.021812230348587036, 0.006957349367439747, -0.013572203926742077, -0.04784185066819191, -0.0067189461551606655, -0.013706515543162823, 0.015513009391725063, -0.019985590130090714, 0.01308196596801281, 0.02185252495110035, -0.009106338024139404, -0.024310430511832237, -0.008663108572363853, -0.17320846021175385, 0.02294044941663742, 0.00661485455930233, -0.010342005640268326, 0.011765711009502411, -0.0004339948936831206, 0.014680275693535805, 0.008132577873766422, -0.01821267604827881, 0.012403691187500954, 0.004341627936810255, -0.01959608681499958, -0.021637625992298126, -0.003488748101517558, 0.022309184074401855, -0.00858252216130495, 0.0045128753408789635, 0.02836664579808712, 0.019703535363078117, 0.006930487230420113, 0.010321859270334244, -0.004620324820280075, 0.00863624643534422, -0.0004688320041168481, 0.006829753518104553, -0.004089793190360069, -0.010939693078398705, 0.014948899857699871, -0.0006749166059307754, -0.018091794103384018, 0.003639848902821541, -0.018024638295173645, 0.026889216154813766, 0.022255459800362587, 0.011053858324885368, -0.020348232239484787, 0.009892061352729797, -0.02338367886841297, 0.0019038693280890584, 0.021691350266337395, 0.03282579407095909, 0.02437758632004261, 0.005620947573333979, -0.004328196868300438, 0.0019491994753479958, 0.00924064964056015, 0.0262579508125782, -0.009495842270553112, 0.008407916873693466, -0.022067423909902573, 0.00616826768964529, -0.03062308207154274, -0.008307183161377907, 0.010268134996294975, 0.0004352540709078312, 0.01578163169324398, -0.01030842773616314, 0.017326217144727707, -0.00979804340749979, -0.011356059461832047, 0.0025468869134783745, -0.030032111331820488, 0.006000378169119358, -0.01707102544605732, -0.013578919693827629, -0.017057593911886215, -0.0062421392649412155, 0.002926317509263754, -0.02745332568883896, 0.009408539161086082, 0.003975628409534693, 0.004170380532741547, 0.02965603768825531, -0.01977069117128849, -0.0013187736039981246, 0.019663242623209953, -0.010724794119596481, 8.331526623805985e-05, 0.007293128874152899, 0.011611252091825008, 0.00742072518914938, 0.017903758212924004, -0.027466755360364914, 0.004331554286181927, -0.0036767846904695034, -0.0006854097591713071, 0.016788970679044724, -0.002857482759281993, -0.008810851722955704, -0.015553302131593227, 0.01642632856965065, -0.024122392758727074, -0.021100377663969994, -0.0060876808129251, 0.00772964209318161, 0.04359759762883186, 0.016977006569504738, -0.0012877141125500202, -0.012645453214645386, -0.01403557974845171, 0.001256654504686594, 0.0009729208541102707, -0.006537625100463629, 0.011141161434352398, 0.029521726071834564, 0.008475072681903839, 0.011973894201219082, 0.009321236982941628, 0.02809802070260048, -0.012135067954659462, -0.012282811105251312, 0.01292079221457243, 0.029548589140176773, 0.032691482454538345, -0.005043406970798969, 0.021865956485271454, 0.0015017734840512276, -0.013155837543308735, -0.006826395634561777, -0.014384790323674679, 0.0372580848634243, 0.011758995242416859, -0.009885345585644245, 0.004553169012069702, -0.01774258352816105, 0.006077607627958059, -0.10637491941452026, -0.023034468293190002, 0.009119768626987934, -0.010080098174512386, -0.00818630214780569, 0.041260574012994766, 0.005533644929528236, 0.017796309664845467, -0.0010425951331853867, 0.026150500401854515, -0.00029380698106251657, -0.020576562732458115, -0.018494730815291405, 0.00012151017290307209, 0.006396597716957331, -0.024511897936463356, -0.03690887242555618, -0.006675294600427151, -0.024632778018712997, 0.024149255827069283, 0.0006740771932527423, -0.03690887242555618, -0.0033275741152465343, -0.013746809214353561, -0.0007697743130847812, -0.007534889969974756, -0.044510919600725174, 0.010617345571517944, 0.01419675350189209, 0.02279270626604557, 0.009274227544665337, -0.020052745938301086, 0.014062441885471344, -0.031751301139593124, -0.0010115355253219604, 0.007004358805716038, -0.026271382346749306, -0.005577296018600464, 0.007528174668550491, -0.031187191605567932, 0.008407916873693466, 0.006977496203035116, 0.014881744049489498, -0.06575904041528702, 0.010610629804432392, -0.013525194488465786, -0.007917678914964199, 0.015875650569796562, 0.02695637196302414, -0.004754636436700821, -0.025680409744381905, 0.01654721051454544, 0.009213787503540516, -0.01605025678873062, 0.005594084970653057, -0.018172381445765495, 0.007152101490646601, -0.01029499713331461, 0.008011696860194206, -0.005641094408929348, -0.014814588241279125, 0.008878007531166077, -0.01233653537929058, -0.003265454899519682, 0.03564634174108505, -0.0037640873342752457, 0.008609384298324585, 0.006463753990828991, 0.02120782807469368, -0.030676808208227158, -0.007131954655051231, 0.010617345571517944, -0.009321236982941628, 0.015996530652046204, -0.0366671122610569, -0.006752524059265852, -0.02933369018137455, -0.005761974956840277, -0.004569957964122295, -0.016238292679190636, -0.008441494777798653, -0.01909913308918476, 0.014559395611286163, -0.029494864866137505, 0.009784611873328686, -0.008448210544884205, -0.013384167104959488, -0.0014153101947158575, -0.009173493832349777, -0.011537380516529083, 0.011215032078325748, 0.024941695854067802, 0.01768885925412178, -0.0010619023814797401, -0.024001512676477432, -0.0007563431281596422, 0.005973516032099724, 0.01590251363813877, 0.05614231899380684, 0.004324838984757662, 0.0007063959492370486, 0.0019844563212245703, -0.052086103707551956, 0.013874405063688755, 0.0014018790097907186, -0.008428063243627548, 0.020804893225431442, -0.0001366202486678958, 0.01322970911860466, -0.02106008492410183, 0.0042375363409519196, -0.007259550970047712, -0.0024931621737778187, 0.0004211933119222522, 0.005248232278972864, 0.0049997554160654545, -0.006420102436095476, -0.020200489088892937, 0.006178341340273619, -0.003110996214672923, 0.03642535209655762, 0.03215423598885536, 0.01563389040529728, 0.008938448503613472, -0.0018921169685199857, -0.006765955127775669, -0.016292016953229904, 0.04238879308104515, -0.039084725081920624, 0.004747921135276556, -0.013612497597932816, -0.046068936586380005, 0.03282579407095909, -0.0021859239786863327, -0.004190526902675629, 0.01900511421263218, -0.005620947573333979, -0.007299844641238451, -0.009717456065118313, 0.018239537253975868, 0.012121637351810932, -0.0005599121795967221, 0.0022967311087995768, -0.02772194892168045, 0.01465341355651617, -0.028232334181666374, -0.0018367134034633636, -0.013914698734879494, -0.023517990484833717, -0.0025049145333468914, 0.02534463070333004, -0.007796797901391983, 0.021758506074547768, 0.015620458871126175, -0.02938741445541382, -0.010180831886827946, -0.012249233201146126, -0.02901134081184864, -0.00371036259457469, -0.0014446909772232175, -0.02349112741649151, -0.013766956515610218, 0.014102735556662083, 0.006198488175868988, 0.024686502292752266, -0.005029975436627865, 0.014210185036063194, -0.004707627464085817, -0.019690105691552162, -0.017353080213069916, 0.02452532947063446, 0.00161677785217762, -0.0011802646331489086, 0.003794307354837656, 0.02358514629304409, -0.016842694953083992, 0.002278263447806239, 0.00913991592824459, 0.0008398682693950832, -0.01642632856965065, -0.006833110935986042, 0.02249722182750702, -0.009724171832203865, -0.012759617529809475, -0.03199306130409241, 0.0032251612283289433, 0.029199378564953804, 0.018884234130382538, -0.023947788402438164, -0.0016302090371027589, -0.012517856433987617, 0.009428686462342739, -0.0013019846519455314, 0.00742072518914938, -0.007494596764445305, 0.010523326694965363, -0.010570336133241653, -0.00164615863468498, -0.0013347232015803456, 0.008515366353094578, -0.008313897997140884, 0.014545964077115059, -0.010644207708537579, -0.0017897043144330382, -0.013592351227998734, -0.026996664702892303, -0.009119768626987934, 0.013834112323820591, -0.017353080213069916, -0.03674769774079323, -0.01622486114501953, 0.025774428620934486, 0.022336047142744064, 0.009327951818704605, 0.006342872977256775, 0.006698799319565296, -0.009079474955797195, 0.006635001394897699, -0.015042917802929878, -0.0004692517395596951, -0.012061196379363537, 0.014680275693535805, 0.00803855899721384, -0.005597442854195833, 0.011503802612423897, 0.006154836621135473, 0.007615477312356234, 0.006957349367439747, 0.012638737447559834, -0.010375583544373512, 0.027533913031220436, 0.0019189793383702636, 0.010167400352656841, -0.024417879059910774, 0.0038581055123358965, -0.01645319163799286, 0.009025750681757927, -0.01637260429561138, 0.0056746723130345345, 0.018320124596357346, -0.026822060346603394, 0.03540458157658577, -0.005234801210463047, -0.0009653658489696681, -0.00390847260132432, 0.026674317196011543, 0.03596869111061096, 0.007709495257586241, 0.015204091556370258, -0.023397110402584076, -0.02314191684126854, -0.0032587391324341297, -0.0038312431424856186, 0.004254325293004513, -0.00016274808149319142, -0.00600709393620491, 0.0128334891051054, 0.0003292736946605146, -0.00039663942879997194, -0.005577296018600464, 0.01369979977607727, 0.013249855488538742, 0.012356682680547237, 0.0012365076690912247, 0.012719323858618736, -0.009542850777506828, -0.007232688833028078, 0.01947520673274994, 0.00013724983728025109, -0.007736357860267162, -0.031133467331528664, 0.02445817179977894, 0.002553602447733283, -0.04201272130012512, -0.01630544848740101, 0.02349112741649151, -0.008354191668331623, -0.008515366353094578, -0.014022148214280605, 0.004489370621740818, 0.01689642108976841, -0.0017410162836313248, 0.0032301980536431074, -0.02839350700378418, -0.013417745940387249, 0.013780387118458748, -0.015647320076823235, -0.017702290788292885, -0.006164910271763802, 0.00362641760148108]

In [ ]:
query = SimpleStatement(
    f"""
    SELECT *
    FROM {ASTRA_DB_KEYSPACE}.{ASTRA_DB_VECTOR_TABLE}
    ORDER BY question_embedding ANN OF {query_vector} LIMIT 10;
    """
    )

results = session.execute(query)
top_answers = results._current_rows

for row in top_answers:
  print(row)